In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=1)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1437, 64), (360, 64), (1437,), (360,))

In [14]:
# 函数式 API 需要 Input 层，返回一个 placeholder 张量
inputs = tf.keras.Input(shape=(64,))
x = tf.keras.layers.Dense(units=30, activation='relu')(inputs)
outputs = tf.keras.layers.Dense(units=10, activation='softmax')(x)

In [15]:
# 函数式 API 需要指定输入和输出
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# 模型编译和训练和顺序模型一致
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=64, epochs=20,
          validation_data=(X_test, y_test))

Train on 1437 samples, validate on 360 samples
Epoch 1/20
1437/1437 [==============================] - 1s 508us/sample - loss: 7.7930 - acc: 0.1441 - val_loss: 5.1425 - val_acc: 0.2639
Epoch 2/20
1437/1437 [==============================] - 0s 40us/sample - loss: 4.1644 - acc: 0.3646 - val_loss: 2.9193 - val_acc: 0.4944
Epoch 3/20
1437/1437 [==============================] - 0s 42us/sample - loss: 2.7070 - acc: 0.5665 - val_loss: 2.0989 - val_acc: 0.6389
Epoch 4/20
1437/1437 [==============================] - 0s 42us/sample - loss: 1.8676 - acc: 0.6806 - val_loss: 1.4192 - val_acc: 0.6361
Epoch 5/20
1437/1437 [==============================] - 0s 40us/sample - loss: 1.1329 - acc: 0.7070 - val_loss: 0.8320 - val_acc: 0.7528
Epoch 6/20
1437/1437 [==============================] - 0s 39us/sample - loss: 0.7164 - acc: 0.8003 - val_loss: 0.6493 - val_acc: 0.7861
Epoch 7/20
1437/1437 [==============================] - 0s 39us/sample - loss: 0.5435 - acc: 0.8344 - val_loss: 0.5175 - val_acc: 

### 关于模型保存
1. 对于较大的训练任务，保存模型可以方便后续恢复重新使用。
2. 保存后的模型可以方便模型部署。

TensorFlow 模型一般包含 3 类要素，分别是：模型权重值、模型配置乃至优化器配置。

如果只需要保存模型权重值，可以使用 `tf.keras.Model.save_weights` <a href="https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#save_weights"><i class="fa fa-external-link-square" aria-hidden="true"></i></a>，并指定存放路径。

In [17]:
model.save_weights('./weights/model')  # 保存检查点名称为 model，路径为 ./weights


Consider using a TensorFlow optimizer from `tf.train`.


In [18]:
!ls './weights/'  # 直接运行查看目录下文件

checkpoint                model.data-00000-of-00001 model.index


In [19]:
model.load_weights('./weights/model')  # 恢复检查点

默认情况下，该方法会以 TensorFlow 检查点文件格式 <a href="https://www.tensorflow.org/guide/checkpoints"><i class="fa fa-external-link-square" aria-hidden="true"></i></a> 保存模型的权重。检查点文件是 TensorFlow 特有的模型权重保存方法，其默认会以每 10 分钟（600 秒）写入一个检查点，训练时间较短则只保存一个检查点。检查点默认情况下只保存 5 个，即模型训练过程中不同时间点的版本状态。

我们一般会在大型任务训练时设置检查点保存。这样做的好处在于一旦因为意外情况导致训练终止，TensorFlow 可以加载检查点状态，避免又需要从头开始训练。

如果我们需要模型推理，一般情况会使用 `model.save` 保存完整的模型，即包含模型权重值、模型配置乃至优化器配置等。例如，下面将模型存为 Keras HDF5 格式，其为 Keras 多后端实现的默认格式。

In [20]:
model.save('model.h5')  # 保存完整模型

In [21]:
model_ = tf.keras.models.load_model('model.h5')  # 调用模型

In [22]:
model_.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 30)                1950      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                310       
Total params: 2,260
Trainable params: 2,260
Non-trainable params: 0
_________________________________________________________________


In [23]:
preds = model_.predict(X_test[:3])  # 预测前 3 个测试样本
preds

array([[7.8036662e-09, 9.9711108e-01, 8.3323027e-04, 6.6171054e-08,
        6.0713497e-05, 5.7099049e-07, 3.2388005e-05, 2.1985049e-07,
        1.9616529e-03, 2.3022364e-09],
       [1.0787126e-09, 6.1167867e-07, 6.8338180e-05, 1.8049786e-08,
        9.3875963e-07, 9.9341053e-01, 5.5667475e-09, 3.4371477e-03,
        3.0472213e-03, 3.5199493e-05],
       [9.9998474e-01, 2.2993392e-09, 8.0985579e-08, 9.7194264e-10,
        3.4024758e-06, 1.5779056e-06, 1.0033179e-05, 2.6977229e-09,
        1.0161685e-07, 1.0134030e-10]], dtype=float32)

In [24]:
np.argmax(preds, axis=1), np.max(preds,axis=1)

(array([1, 5, 0]), array([0.9971111 , 0.9934105 , 0.99998474], dtype=float32))

In [27]:
y_test[:3]

array([1, 5, 0])

一般情况下，模型的预测是正确的。如果错误可以检查上方模型的评估情况，合理增大 Epoch 以提高模型准确度。